In [2]:
import numpy as np
import pandas as pd

In [14]:
def num_to_chair(arr):
    x = str(arr[0]+1)
    y = arr[1]
    y = chr(y+65)
    return x+y

In [15]:
def chair_to_num(arr):
    x = int(arr[:-1])-1
    y = arr[-1]
    y = ord(y)-65
    return (x, y)

In [45]:
def get_min_dist(vectors):
    combs = [(vectors[i], vectors[j]) for i in range(len(vectors)) for j in range(i+1,len(vectors))]
    from scipy.spatial import distance
    values = list()
    for comb in combs:
        values.append(distance.euclidean(comb[0], comb[1]))
    return min(values)

In [55]:
def assign_flight(df, DepartureDate, FlightNumber):
    # Filter date
    filtered_df = df[(df['DepartureDate'] == DepartureDate) & (df['FlightNumber'] == FlightNumber)]
    filtered_df = filtered_df.sort_values(['SeatBookDate', 'SeatBookDateTime', 'BookingBookDate'])
    filtered_df['AssignedSeat'] = ''

    # Get costs
    cost = np.zeros((32, 6))
    cost[0,:] = 39
    cost[0,[1,-2]] = 34
    cost[1:5,:] = 34
    cost[1:5,[1,-2]] = 29
    cost[5:11,:] = 27
    cost[5:11,[1,-2]] = 22
    cost[11:13,:] = 29
    cost[11:13,[1,-2]] = 24
    cost[13:23,:] = 18
    cost[13:23,[1,-2]] = 12
    cost[23:,:] = 14
    cost[23:,[1,-2]] = 9
    cost[-1,2:] = 99999

    # Asigna los que compraron silla
    chairs = list(filtered_df[~filtered_df['SeatBookDate'].isna()]['UnitDesignator'])
    passengers = list(filtered_df[~filtered_df['SeatBookDate'].isna()]['PassengerID'])
    for i in range(len(chairs)):
        chair = chairs[i]
        passenger = passengers[i]
        cost[(chair_to_num(chair))] = passenger
        filtered_df.loc[filtered_df['PassengerID'] == passenger, 'AssignedSeat'] = chair
    
    # Asigna el resto
    bookings = list(filtered_df[filtered_df['SeatBookDateTime'].isna()]['RecordLocator'])
    booking_set = set(bookings)
    total_cost = 0
    for booking in booking_set:
        passengers = filtered_df.loc[(filtered_df['SeatBookDateTime'].isna()) & (filtered_df['RecordLocator'] == booking), 'PassengerID']
        chairs = list()
        for passenger in passengers:
            min_cost = cost.min()
            position = (np.asarray(cost == min_cost).nonzero()[0][0], np.asarray(cost == min_cost).nonzero()[1][0])
            last_position = (np.asarray(cost == min_cost).nonzero()[0][-1], np.asarray(cost == min_cost).nonzero()[1][-1])
            aux = chairs.copy()
            aux.append(last_position)
            # aux = list(aux)
            aux = np.array(aux)
            if len(chairs) == 0:
                chair = num_to_chair(position)
                total_cost += cost[(chair_to_num(chair))]
                cost[(chair_to_num(chair))] = passenger
                filtered_df.loc[filtered_df['PassengerID'] == passenger, 'AssignedSeat'] = chair
                chairs.append(position)
            elif get_min_dist(aux) > 1.5:
                chair = num_to_chair(last_position)
                total_cost += cost[(chair_to_num(chair))]
                cost[(chair_to_num(chair))] = passenger
                filtered_df.loc[filtered_df['PassengerID'] == passenger, 'AssignedSeat'] = chair
                chairs.append(last_position)
            elif np.asarray(cost == min_cost).nonzero()[0].shape[0] > 2:
                for i in range(np.asarray(cost == min_cost).nonzero()[0].shape[0]):
                    test_position = (np.asarray(cost == min_cost).nonzero()[0][i], np.asarray(cost == min_cost).nonzero()[1][i])
                    aux = chairs
                    aux.append(test_position)
                    aux = list(aux)
                    aux = np.array(aux)
                    if get_min_dist(aux) > 1.5:
                        chair = num_to_chair(test_position)
                        total_cost += cost[(chair_to_num(chair))]
                        cost[(chair_to_num(chair))] = passenger
                        filtered_df.loc[filtered_df['PassengerID'] == passenger, 'AssignedSeat'] = chair
                        chairs.append(test_position)
                        break
            else:
                flag = False
                for min_cost in np.unique(cost[(cost<1000)&(cost>min_cost)]):
                    position = (np.asarray(cost == min_cost).nonzero()[0][0], np.asarray(cost == min_cost).nonzero()[1][0])
                    last_position = (np.asarray(cost == min_cost).nonzero()[0][-1], np.asarray(cost == min_cost).nonzero()[1][-1])
                    aux = chairs
                    aux.append(test_position)
                    aux = list(aux)
                    aux = np.array(aux)
                    if get_min_dist(aux) > 1.5:
                        chair = num_to_chair(last_position)
                        total_cost += cost[(chair_to_num(chair))]
                        cost[(chair_to_num(chair))] = passenger
                        filtered_df.loc[filtered_df['PassengerID'] == passenger, 'AssignedSeat'] = chair
                        chairs.append(last_position)
                    elif np.asarray(cost == min_cost).nonzero()[0].shape[0] > 2:
                        for i in range(np.asarray(cost == min_cost).nonzero()[0].shape[0]):
                            test_position = (np.asarray(cost == min_cost).nonzero()[0][i], np.asarray(cost == min_cost).nonzero()[1][i])
                            aux = chairs
                            aux.append(test_position)
                            aux = list(aux)
                            aux = np.array(aux)
                            if get_min_dist(aux) > 1.5:
                                chair = num_to_chair(test_position)
                                total_cost += cost[(chair_to_num(chair))]
                                cost[(chair_to_num(chair))] = passenger
                                filtered_df.loc[filtered_df['PassengerID'] == passenger, 'AssignedSeat'] = chair
                                chairs.append(test_position)
                                flag = True
                                break
                        
                        if flag:
                            break
                        

    # Result to Excel
    with pd.ExcelWriter(
        "output.xlsx",
        mode="a",
        engine="openpyxl",
        if_sheet_exists="replace",
    ) as writer:
        filtered_df.to_excel(writer, sheet_name=f"{FlightNumber}, {DepartureDate}")
    
    return total_cost

In [54]:
np.unique(cost[(cost<1000)&(cost>9)])

array([12., 14., 18., 22., 24., 27., 29., 34., 39.])

In [50]:
cost.

array([[3.9000e+01, 3.4000e+01, 3.9000e+01, 3.9000e+01, 3.4000e+01,
        3.9000e+01],
       [3.4000e+01, 2.9000e+01, 3.4000e+01, 3.4000e+01, 2.9000e+01,
        3.4000e+01],
       [3.4000e+01, 2.9000e+01, 3.4000e+01, 3.4000e+01, 2.9000e+01,
        3.4000e+01],
       [3.4000e+01, 2.9000e+01, 3.4000e+01, 3.4000e+01, 2.9000e+01,
        3.4000e+01],
       [3.4000e+01, 2.9000e+01, 3.4000e+01, 3.4000e+01, 2.9000e+01,
        3.4000e+01],
       [2.7000e+01, 2.2000e+01, 2.7000e+01, 2.7000e+01, 2.2000e+01,
        2.7000e+01],
       [2.7000e+01, 2.2000e+01, 2.7000e+01, 2.7000e+01, 2.2000e+01,
        2.7000e+01],
       [2.7000e+01, 2.2000e+01, 2.7000e+01, 2.7000e+01, 2.2000e+01,
        2.7000e+01],
       [2.7000e+01, 2.2000e+01, 2.7000e+01, 2.7000e+01, 2.2000e+01,
        2.7000e+01],
       [2.7000e+01, 2.2000e+01, 2.7000e+01, 2.7000e+01, 2.2000e+01,
        2.7000e+01],
       [2.7000e+01, 2.2000e+01, 2.7000e+01, 2.7000e+01, 2.2000e+01,
        2.7000e+01],
       [2.9000e+01, 2

In [21]:
A = np.array([[0, 0], [0, 1], [0, 3], [1, 2], [2, 2]])
B = np.array([[ 0.1,  0.4,  0.5],
    [ 0.7,  0.0,  0.4],
    [ 0.8,  0.4,  0.7],
    [ 0.9,  0.3,  0.8]])

In [35]:
a = (1,1)
b = (2,3)
c = (5,5)
test = list()
test.append(a)
test.append(b)
test.append(c)

In [36]:
test

[(1, 1), (2, 3), (5, 5)]

In [39]:
combs = [(test[i], test[j]) for i in range(len(test)) for j in range(i+1,len(test))]
combs

[((1, 1), (2, 3)), ((1, 1), (5, 5)), ((2, 3), (5, 5))]

In [42]:
for comb in combs:
    print(comb)

((1, 1), (2, 3))
((1, 1), (5, 5))
((2, 3), (5, 5))


In [41]:
combs[1][0]

(1, 1)

2.23606797749979

In [12]:
df = pd.read_excel('BD SEAT .xlsx')
df.shape

(55600, 10)

In [49]:
assign_flight(df, '2022-06-26', 56261)

1322.0